
<center>  
 John Ndolo
    
 
   <center>
 Assignment 1
 </center>

# 1. Grading criteria

1. Correctness of code (60%).
2. Quality and efficiency of the code (30%) --> Can scale and tackle edge cases.
3. Presentation of the document (10%) --> Readability and intuitiveness.

# 2. Problem 1

Assume you have two tables:

Table 1: student names.

| student_id      | first_name     | last_name      |
| --------------- | ---------------| ---------------|
| 1               | Alice          | Jones          |
| 2               | Andy           | Jackson        |

Table 2: student grades.

| student_id      | date           | gpa            |
| --------------- | ---------------| ---------------|
| 1               | 2022-01-01     | NULL           |
| 1               | 2022-04-01     | 0.80           |
| 1               | 2022-08-01     | 0.83           |
| 2               | 2021-01-01     | 0.83           |
| 2               | 2021-04-01     | 0.84           |
| 2               | 2021-08-01     | 0.82           |

The goal is to query the above tables and produce the following table (ordered by student_id):

| student_id      | full_name      | most_recent_gpa|
| --------------- | ---------------| ---------------|
| 1               | Alice Jones    | 0.83           |
| 2               | Andy  Jackson  | 0.82           |

Tasks:

1. Create a Jupyter Notebook that creates the above input tables using pandas.
2. In Jupyter, use pandas to create the output table.
3. In Jupyter, use sqlite3 to create the database and the output table.

# Solution to Problem 1

In [4]:
#!pip install Levenshtein

### 1. importing dependence libraries

In [1]:
import os
import numpy as np
import pandas as pd

import sqlite3
database_path = ""

import re
from Levenshtein import ratio
import requests
import io

import warnings
warnings.filterwarnings('ignore')


In [2]:
%%capture
%load_ext sql

### 2. Pandas dataframes

#### Task1 : Jupyter Notebook that creates the above input tables using pandas

In [3]:
# Table 1: student names
data = {'student_id': [1, 2],
        'first_name': ['Alice', 'Andy'],
        'last_name': ['Jones', 'Jackson']}

df_student_names = pd.DataFrame(data)
df_student_names

,student_id,first_name,last_name
0,1,Alice,Jones
1,2,Andy,Jackson


In [4]:
# Table 2: student grades
data = {'student_id': [1, 1, 1, 2, 2, 2],
        'date': ['2022-01-01', '2022-04-01', '2022-08-01', '2021-01-01', '2021-04-01', '2021-08-01'],
        'gpa': ['NULL', 0.80, 0.83, 0.83, 0.84, 0.82]}

df_student_grades = pd.DataFrame(data)
df_student_grades

,student_id,date,gpa
0,1,2022-01-01,NULL
1,1,2022-04-01,0.8
2,1,2022-08-01,0.83
3,2,2021-01-01,0.83
4,2,2021-04-01,0.84
5,2,2021-08-01,0.82


#### Task2 : using pandas to create the output table.

In [6]:
# Join tables on student_id and select most recent gpa 
output_table = (pd.merge(df_student_names, df_student_grades, on='student_id')
                .sort_values(by=['student_id', 'date'], ascending=[True, False])
                .drop_duplicates(subset='student_id'))

output_table.reset_index(drop = True)

,student_id,first_name,last_name,date,gpa
0,1,Alice,Jones,2022-08-01,0.83
1,2,Andy,Jackson,2021-08-01,0.82


In [8]:
# Add full_name column
output_table['full_name'] = output_table['first_name'] + ' ' + output_table['last_name']

output_table.reset_index(drop = True)

,student_id,first_name,last_name,date,gpa,full_name
0,1,Alice,Jones,2022-08-01,0.83,Alice Jones
1,2,Andy,Jackson,2021-08-01,0.82,Andy Jackson


In [9]:
# Select only necessary columns and reorder them
output_table = output_table[['student_id', 'full_name', 'gpa']].rename(columns = {'gpa':'most_recent_gpa'})
output_table.reset_index(drop=True)

,student_id,full_name,most_recent_gpa
0,1,Alice Jones,0.83
1,2,Andy Jackson,0.82


#### Task3: Using sqlite3 to create the database and the output table.

In [10]:
if os.path.isfile("students.db"):
    os.remove("students.db")
conn = sqlite3.connect("students.db")
df_student_names.to_sql("student_names", conn, if_exists="replace", index=False)
df_student_grades.to_sql("student_grades", conn, if_exists="replace", index=False)
#df_student_degrees.to_sql("student_degrees", conn, if_exists="replace", index=False)
conn.close()

In [11]:
%%sql sqlite:///students.db
/* selecting the entire table*/
SELECT * FROM STUDENT_NAMES;

Done.


student_id,first_name,last_name
1,Alice,Jones
2,Andy,Jackson


In [12]:
%%sql sqlite:///students.db
/* selecting the entire table*/
SELECT * FROM STUDENT_GRADES;

Done.


student_id,date,gpa
1,2022-01-01,NULL
1,2022-04-01,0.8
1,2022-08-01,0.83
2,2021-01-01,0.83
2,2021-04-01,0.84
2,2021-08-01,0.82


In [13]:
%%sql sqlite:///students.db

SELECT student_names.student_id, 
       first_name || ' ' || last_name as full_name, 
       (SELECT gpa FROM student_grades WHERE student_id = student_names.student_id ORDER BY date DESC) #DESC--->descending 
        as most_recent_gpa
        
FROM student_names
ORDER BY student_names.student_id

Done.


student_id,full_name,most_recent_gpa
1,Alice Jones,0.83
2,Andy Jackson,0.82


# 3. Problem 2

You are tasked to do a fuzzy string matching exercise for a start-up. You are given an internal CSV file "internal_company_data.csv" and an external file (source: https://open.canada.ca/data/en/dataset/c1bb15bd-0b4d-484c-a813-857b167d8d00 Links to an external site) "regulated_company_list.csv".

Using the Levenshtein ratio (info: https://en.wikipedia.org/wiki/Levenshtein_distance Links to an external site.; library: https://pypi.org/project/python-Levenshtein/ Links to an external site.), write a function in Python that provides the closest matches for internal company to those found in the regulated-company-list. Include the following fields:

| id              | company_name_internal | company_name_closest_match | score   | accountable_officer |
| --------------- | --------------------- | -------------------------- | --------| ------------------- |
|                 |                       |                            |         |                     |
|                 |                       |                            |         |                     |


Before this function, write a function that cleans company name (internal and regulated) in the following manner:

1. Convert all characters to lower case.

2. Punctuation: Remove the following characters = [".",  ","]

3. Stop words: Remove the following words from the name ["corporation", "limited", "ltd", "inc", "national", "international", "company", "ulc"]

Examples --> clean("AAaa123") == "aaaa123"; clean("A corporation limited ltd, national international, company, ulc. boy") == "a boy"


Once done, feed this into the Levenshtein score.

# Solution to problem 2

### Task 1: Loading the datasets

In [27]:
#Internal dataset
internal_company_data = pd.read_csv('internal_company_data.csv')
internal_company_data.head()

,id,company_name
0,1,Foothills Pipe Lines
1,2,PKM Cochin
2,3,Maritimes and Northeast Pipeline
3,4,NOVA Gas Transmission
4,5,Trans Mountain Pipeline


In [28]:
#extrenal dataset
path = "https://www.cer-rec.gc.ca/open/adjudication/regulated-company-list.csv"
regulated_company_list= pd.read_csv(path, encoding='ISO-8859-1')
regulated_company_list.head()

,Company,Company Website,Contact Information,Accountable Officer
0,Group 1,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,Alliance Pipeline Ltd.,https://www.alliancepipeline.com/Pages/default...,NaN,Cynthia Hansen
3,Enbridge Pipelines (NW) Inc.,https://www.enbridge.com/,NaN,Colin Gruending
4,Enbridge Pipelines Inc.,https://www.enbridge.com/,NaN,Colin Gruending


### Task 2: The function to clean the datasets

In [29]:
def clean(name):
    if type(name)==str:
        # Convert to lower case
        name = name.lower()
        # Remove punctuation
        name = re.sub(r'[.,]', '', name)
        # Remove stop words
        stop_words = ["corporation", "limited", "ltd", "inc", "national", "international", "company", "ulc"]
        name = " ".join([word for word in name.split() if word not in stop_words])
    return name


In [30]:
#Example
clean("A corporation limited ltd, national international, company, ulc. boy")

'a boy'

### Task 3: Function providing the closest matches for internal company to those  in the regulated-company-list

In [31]:
def closest_match(internal_file, regulated_file):
    internal_file["company_name"] = internal_file["company_name"].apply(clean)
    regulated_file["Company"] = regulated_file["Company"].apply(clean)
    matches_df = pd.DataFrame(columns=["id", "company_name_internal",
                                       "company_name_closest_match", "score", "accountable_officer"])
    
    for index, row in internal_file.iterrows():
        closest_match = ""
        closest_score = 0 #assumming no match as an initialization
        closest_officer = ""
        for index2, row2 in regulated_file.iterrows():
            score = ratio(row["company_name"], row2["Company"])
            if score > closest_score:
                closest_score = score
                closest_match = row2["Company"]
                closest_officer = row2["Accountable Officer"]
        matches_df = matches_df.append(
            {
                "id": row["id"],
                "company_name_internal": row["company_name"],
                "company_name_closest_match": closest_match,
                "score": closest_score,
                "accountable_officer": closest_officer
            },
            ignore_index=True
        )
    return matches_df


In [32]:
matches_df = closest_match(internal_company_data, regulated_company_list)
print('Shape of the Matched dataframe: ',matches_df.shape)
matches_df.head()

Shape of the Matched dataframe:  (20, 5)


,id,company_name_internal,company_name_closest_match,score,accountable_officer
0,1,foothills pipe lines,foothills pipe lines,1.000000,Jawad Masud
1,2,pkm cochin,pkm cochin,1.000000,Jaret Sprott
2,3,maritimes and northeast pipeline,maritimes & northeast pipeline management,0.794521,Cynthia Hansen
3,4,nova gas transmission,nova gas transmission,1.000000,Jawad Masud
4,5,trans mountain pipeline,trans mountain pipeline,1.000000,Rob Van Walleghem
